In [16]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")   
classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9598051905632019},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

# How transformers works

In [17]:
# Get the tokenizer from the same pretained model

from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
# Convert text into tokens and return in tensor format as transformers take tensor as input

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [19]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [20]:
print(outputs.logits)
print(outputs.logits.shape)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
torch.Size([2, 2])


In [21]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4419e-04]], grad_fn=<SoftmaxBackward0>)


In [22]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [23]:
# model.save_pretrained(".")   // For saving a model in local

# How Tokenizer works

In [24]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Encoding

In [25]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
# This is Subword tokenization
tokens= tokenizer.tokenize(raw_inputs) 
print(tokens)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', 'i', 'hate', 'this', 'so', 'much', '!']


In [26]:
ids= tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 1045, 5223, 2023, 2061, 2172, 999]


## Decoding

In [27]:
decoded_strings= tokenizer.decode(ids)
print(decoded_strings)

i've been waiting for a huggingface course my whole life. i hate this so much!


## Putting all together

In [28]:
# Get the tokenizer from the same pretained model as it was trained

from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [31]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
# convert sequences into words (Subword tokenization)
tokens= tokenizer.tokenize(raw_inputs) 
print("tokens",tokens)

# convert tokens to ids
ids= tokenizer.convert_tokens_to_ids(tokens)
print("ids",ids)

tokens ['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', 'i', 'hate', 'this', 'so', 'much', '!']
ids [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 1045, 5223, 2023, 2061, 2172, 999]


In [44]:
import torch

# conver ids to tensor as model accepts tensor
# Transformers models expect multiple sentences by default so adding extra [] dimension
input_ids= torch.tensor([ids]);
print("input ids",input_ids)

input ids tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012,  1045,  5223,  2023,  2061,  2172,   999]])


In [47]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(input_ids)
print("outputs",outputs.logits)

outputs tensor([[-2.7657,  2.8486]], grad_fn=<AddmmBackward0>)


- **Padding** is added as tensor needs to be rectamgular shape and if in batching process we pass multiple sequences with different length it will not work.
    ```
    # Example:
    
    # This will not work
    batched_ids = [
        [200, 200, 200],
        [200, 200]
    ]

    # Now we add a padding to make it same length
    batched_ids = [
        [200, 200, 200],
        [200, 200, tokenizer.pad_token_id],
    ]

    ```

- Now since we have added padding to make rectangular shape but we don't want to consider it as context will change based on that so to ignore it we add **Attention masks** which have same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to.

    ```
        batched_ids = [
        [200, 200, 200],
        [200, 200, tokenizer.pad_token_id],
    ]

    attention_mask = [
        [1, 1, 1],
        [1, 1, 0],
    ]

    outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
    ```

    Now this will work fine as expected.